In [1]:
# importing the dependencies
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [2]:
# 5 emotions considered
num_of_classes = 5
image_rows = 48
image_cols = 48
batch_size = 32

In [3]:
train_data_generate = ImageDataGenerator(
    rotation_range = 30, # in any direction, rotation of image is atmost 30 degrees
    rescale = 1./255, #normalizing pixels of the image - to help neurons avoid bias 
    shear_range = 0.3, 
    zoom_range = 0.3, #the extent to which zoom is applied
    width_shift_range = 0.4, 
    height_shift_range = 0.4, 
    horizontal_flip = True, #emotion does not change for the mirror image  
    fill_mode = "nearest"
)

In [4]:
validation_data_generate = ImageDataGenerator(
    rescale = 1./255, #normalizing pixels of the image - to help neurons avoid bias 
)

In [5]:
train_data_directory = "dataset/train"

In [6]:
validation_data_directory = "dataset/validation"

In [7]:
train_data = train_data_generate.flow_from_directory(
    train_data_directory, 
    color_mode = "grayscale",
    target_size = (image_rows, image_cols), 
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle = True 
)

Found 24256 images belonging to 5 classes.


In [8]:
validation_data = validation_data_generate.flow_from_directory(
    validation_data_directory, 
    color_mode = "grayscale",
    target_size = (image_rows, image_cols), 
    batch_size = batch_size,
    class_mode = "categorical",
    shuffle = True
)

Found 3006 images belonging to 5 classes.


In [9]:
# defining the model
model = Sequential()

In [ ]:
#Block-1
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(image_rows,image_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',
                 input_shape=(image_rows,image_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-2
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

#Block-3
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-4
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
#Block-5
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Block-6
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
#Block-7
model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 32)        320       
                                                                 
 activation (Activation)     (None, 48, 48, 32)        0         
                                                                 
 batch_normalization (Batch  (None, 48, 48, 32)        128       
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        9248      
                                                                 
 activation_1 (Activation)   (None, 48, 48, 32)        0         
                                                                 
 batch_normalization_1 (Bat  (None, 48, 48, 32)        128       
 chNormalization)                                       

In [12]:
from keras.optimizers import Adam
model.compile(
    loss = "categorical_crossentropy", 
    optimizer = Adam(learning_rate = 0.001), 
    metrics = ['accuracy', 'Precision', 'Recall']
     )

In [13]:
num_of_training_samples = 24256
num_of_validation_samples = 3006

In [14]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [15]:
checkpoint = ModelCheckpoint('EmotionDetectionModel.h5',
                             monitor='val_loss',
                             mode='min',
                             save_best_only=True,
                             verbose=1)

earlystop = EarlyStopping(monitor='val_loss',
                          min_delta=0,
                          patience=3,
                          verbose=1,
                          restore_best_weights=True
                          )

reduce_lr = ReduceLROnPlateau(monitor='val_loss',
                              factor=0.2, #new learning rate = old learning rate*factor - this new learning rate comes into the picture after 3 epochs of no improvement
                              patience=3,
                              verbose=1,
                              min_delta=0.0001)

callbacks = [earlystop,checkpoint,reduce_lr]

In [16]:
model.fit(
    train_data,
    validation_data = validation_data,
    validation_steps = num_of_validation_samples//batch_size, 
    steps_per_epoch = num_of_training_samples//batch_size,
    epochs = 15,
    callbacks = callbacks
)

Epoch 1/15
758/758 [==============================] - ETA: 0s - loss: 1.8991 - accuracy: 0.2350 - precision: 0.2287 - recall: 0.0644
Epoch 1: val_loss improved from inf to 1.55538, saving model to EmotionDetectionModel.h5
758/758 [==============================] - 1115s 1s/step - loss: 1.8991 - accuracy: 0.2350 - precision: 0.2287 - recall: 0.0644 - val_loss: 1.5554 - val_accuracy: 0.2930 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - lr: 0.0010
Epoch 2/15


C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


758/758 [==============================] - ETA: 0s - loss: 1.5795 - accuracy: 0.2822 - precision: 0.2955 - recall: 0.0027
Epoch 2: val_loss improved from 1.55538 to 1.54696, saving model to EmotionDetectionModel.h5
758/758 [==============================] - 230s 303ms/step - loss: 1.5795 - accuracy: 0.2822 - precision: 0.2955 - recall: 0.0027 - val_loss: 1.5470 - val_accuracy: 0.3001 - val_precision: 0.5000 - val_recall: 0.0010 - lr: 0.0010
Epoch 3/15
758/758 [==============================] - ETA: 0s - loss: 1.5486 - accuracy: 0.3016 - precision: 0.4430 - recall: 0.0014
Epoch 3: val_loss improved from 1.54696 to 1.51818, saving model to EmotionDetectionModel.h5
758/758 [==============================] - 231s 304ms/step - loss: 1.5486 - accuracy: 0.3016 - precision: 0.4430 - recall: 0.0014 - val_loss: 1.5182 - val_accuracy: 0.3142 - val_precision: 0.5000 - val_recall: 6.7204e-04 - lr: 0.0010
Epoch 4/15
758/758 [==============================] - ETA: 0s - loss: 1.5201 - accuracy: 0.3272